In [1]:

import pandas as pd

import typing as tt
import numpy as np
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import product
from tqdm import tqdm

In [2]:
df=pd.read_csv('../EUR_USD_2020_M5.csv')

In [3]:
df=df.set_index('time')

In [ ]:
class EMD():

    def __init__(self,data):

        self.data=data
        self.possible_pos={'m_a_ap_b_av':{'name':'mean_above_avg_peak_below_avg_valley',
                           'positions':{'buy':'mean_above_avg_peak','sell':'mean_below_avg_valley'},
                           'strategy_desc':'Strategy EMD (EmpiricalModeDecomposition by J.Ehlers) change position above below given levels'}}
        